## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()          

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://2176a0f0-3503-11ed-be64-b1a4781e5f0a.cloud.hopsworks.ai/p/128
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;">🪄 Retrieving Feature Groups</span>

In [2]:
profile_fg = fs.get_or_create_feature_group(
    name = 'profile_fg',
    version = 1
)
tickets_fg = fs.get_or_create_feature_group(
    name = 'tickets_fg',
    version = 1
)

## <span style="color:#ff5f27;">💼 Query Preparation</span>

In [3]:
query = profile_fg.select_all().join(tickets_fg.select_all())

query_show = query.show(5)
col_names = query_show.columns

query_show

2022-09-16 11:54:44,790 INFO: USE `model_serving_featurestore`
2022-09-16 11:54:45,527 INFO: SELECT `fg1`.`passengerid` `passengerid`, `fg1`.`sex` `sex`, `fg1`.`age` `age`, `fg1`.`survived` `survived`, `fg0`.`pclass` `pclass`, `fg0`.`sibsp` `sibsp`, `fg0`.`parch` `parch`, `fg0`.`fare` `fare`, `fg0`.`embarked` `embarked`
FROM `model_serving_featurestore`.`profile_fg_1` `fg1`
INNER JOIN `model_serving_featurestore`.`tickets_fg_1` `fg0` ON `fg1`.`passengerid` = `fg0`.`passengerid`


,passengerid,sex,age,survived,pclass,sibsp,parch,fare,embarked
0,451,male,36.0,0,2,1,2,27.7500,S
1,299,male,28.0,1,1,0,0,30.5000,S
2,352,male,29.0,0,1,0,0,35.0000,S
3,497,female,54.0,1,1,1,0,78.2667,C
4,690,female,15.0,1,1,0,1,211.3375,S


## <span style="color:#ff5f27;">🧑🏻‍🔬 Transformation functions</span>

In [4]:
standard_scaler = fs.get_transformation_function(name = 'standard_scaler')
min_max_scaler = fs.get_transformation_function(name = 'min_max_scaler')
label_encoder = fs.get_transformation_function(name = 'label_encoder')

In [5]:
mapping_transformers = {col_name:label_encoder for col_name in ['sex','embarked']}
mapping_transformers.update({col_name:standard_scaler for col_name in ['pclass','sibsp','parch','age']})
mapping_transformers.update({'fare':min_max_scaler})

mapping_transformers

{'sex': <hsfs.transformation_function.TransformationFunction at 0x7f7673d18a00>,
 'embarked': <hsfs.transformation_function.TransformationFunction at 0x7f7673d18a00>,
 'pclass': <hsfs.transformation_function.TransformationFunction at 0x7f76beb3cdf0>,
 'sibsp': <hsfs.transformation_function.TransformationFunction at 0x7f76beb3cdf0>,
 'parch': <hsfs.transformation_function.TransformationFunction at 0x7f76beb3cdf0>,
 'age': <hsfs.transformation_function.TransformationFunction at 0x7f76beb3cdf0>,
 'fare': <hsfs.transformation_function.TransformationFunction at 0x7f7673d18430>}

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

In [6]:
feature_view = fs.create_feature_view(
    name='titanic_fv',
    version=1,
    transformation_functions=mapping_transformers,
    query=query,
    labels=['survived']
)

Feature view created successfully, explore it at 
https://2176a0f0-3503-11ed-be64-b1a4781e5f0a.cloud.hopsworks.ai/p/128/fs/76/fv/titanic_fv/version/1


## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>



In [7]:
feature_view.create_training_data()

Training dataset job started successfully, you can follow the progress at 
https://2176a0f0-3503-11ed-be64-b1a4781e5f0a.cloud.hopsworks.ai/p/128/jobs/named/titanic_fv_1_1_create_fv_td_16092022115512/executions


(1, <hsfs.core.job.Job at 0x7f7673d41730>)

In [8]:
feature_view.create_train_validation_test_split(
    validation_size=0.1,
    test_size=0.2
)

Training dataset job started successfully, you can follow the progress at 
https://2176a0f0-3503-11ed-be64-b1a4781e5f0a.cloud.hopsworks.ai/p/128/jobs/named/titanic_fv_1_2_create_fv_td_16092022115618/executions


(2, <hsfs.core.job.Job at 0x7f7673cd3670>)

---